In [ ]:
import os
import sys

def download(name, model, iters):
    trained_policy = '{}-{}.iter={}.npz'.format(name, model, iters)
    if not os.path.exists('.policy/{}'.format(trained_policy)):
        print 'download the model from HPC server'
        os.system('scp mercer:/scratch/jg5223/exp/.policy/{} .policy/'.format(trained_policy))
    print 'done'
# m ='model_wmt15_bpe2k_uni_en-ru_rev.npz'    
# download('160920-225547', m, '44400')
# download('160920-225339', m, '33600') 
# download('160920-230514', m, '95600')
# download('160920-225917', m, '68000')
# download('160920-230503', m, '48200')
# download('160920-225454', m, '45200')

# m ='model_wmt15_bpe2k_uni_en-de_rev.npz' 
# download('160925-000643', m, '24400')
# download('160925-000657', m, '10400')
# download('160925-000642', m, '8000') 
# download('160925-000735', m, '22200')
# download('160925-000751', m, '26200')
# download('160925-000811', m, '18800')

# max model 172 160917-175503 Iter=34400 AP=0.3 0.14121125914
# max model 262 160917-001117 Iter=52400 AP=0.5 0.157379802964
# max model 141 160917-175423 Iter=28200 AP=0.7 0.15857511208
# max model 67 160917-175657 Iter=13400 CW=5.0 0.160038686998
m ='model_wmt15_bpe2k_uni_en-de.npz' 
download('160917-175503', m, '34400')
download('160917-001117', m, '52400')
download('160917-175423', m, '28200') 
download('160917-175657', m, '13400')



In [ ]:
# %load weaver_eval.py
"""
Simultaneous Machine Translateion: Training with Policy Gradient

"""
import argparse
import os
import cPickle as pkl

from bleu import *
from nmt_uni import *
from policy import Controller as Policy
from utils import Progbar, Monitor

from simultrans_beam import simultaneous_decoding as simultaneous_beamsearch
from simultrans_model import simultaneous_decoding
from simultrans_model import _seqs2words, _bpe2words, _action2delay, PIPE, _padding

import time

numpy.random.seed(19920206)
timer = time.time

WORK  = '/misc/kcgscratch1/ChoGroup/thoma_exp/SimulTrans/'
EXP   = WORK

# check hidden folders
def check_env():
    import os
    paths = ['.policy', '.pretrained', '.log', '.config', '.images', '.translate']
    for p in paths:
        p = WORK + p
        if not os.path.exists(p):
            os.mkdir(p)

policy = OrderedDict()
policy['prop']      = 0.5
policy['recurrent'] = True
policy['layernorm'] = False
policy['updater']   = 'REINFORCE'
policy['act_mask']  = True

config = OrderedDict()
config['step']      = 1
config['peek']      = 1
config['s0']        = 1
config['sample']    = 1
config['batchsize'] = 1
config['target']    = 1
config['gamma']     = 10
config['Rtype']     = 10
config['forget']    = False
config['maxsrc']    = 5
config['pre']       = False
config['coverage']  = True
config['upper']     = False
config['finetune']  = 'nope'
            
model        = 'model_wmt15_bpe2k_uni_en-de_rev.npz'
options_file = None
id           = None
check_env()

if id is not None:
    fcon = WORK + '.config/{}.conf'.format(id)
    if os.path.exists(fcon):
        print 'load config files'
        policy, config = pkl.load(open(fcon, 'r'))

# ============================================================================== #
# load model model_options
# ============================================================================== #
_model = model
model = WORK + '.pretrained/{}'.format(model)

if options_file is not None:
    with open(options_file, 'rb') as f:
        options = pkl.load(f)
else:
    with open('%s.pkl' % model, 'rb') as f:
        options = pkl.load(f)

print 'load options...'
for w, p in sorted(options.items(), key=lambda x: x[0]):
    print '{}: {}'.format(w, p)

print '***' * 10
print 'load policy options...'
for w, p in sorted(policy.items(), key=lambda x: x[0]):
    print '{}: {}'.format(w, p)    

print '***' * 10
print 'load learning configs...'
for w, p in sorted(config.items(), key=lambda x: x[0]):
    print '{}: {}'.format(w, p)    

# load detail settings from option file:
dictionary, dictionary_target = options['dictionaries']

def _iter(fname):
    with open(fname, 'r') as f:
        for line in f:
            words = line.strip().split()
            x = map(lambda w: word_dict[w] if w in word_dict else 1, words)
            x = map(lambda ii: ii if ii < options['n_words'] else 1, x)
            x += [0]
            yield x

def _check_length(fname):
    f = open(fname, 'r')
    count = 0
    for _ in f:
        count += 1
    f.close()

    return count

# load source dictionary and invert
with open(dictionary, 'rb') as f:
    word_dict = pkl.load(f)
word_idict = dict()
for kk, vv in word_dict.iteritems():
    word_idict[vv] = kk
word_idict[0] = '<eos>'
word_idict[1] = 'UNK'

# load target dictionary and invert
with open(dictionary_target, 'rb') as f:
    word_dict_trg = pkl.load(f)
word_idict_trg = dict()
for kk, vv in word_dict_trg.iteritems():
    word_idict_trg[vv] = kk
word_idict_trg[0] = '<eos>'
word_idict_trg[1] = 'UNK'

## use additional input for the policy network
options['pre'] = config['pre']
print 'setting done.'


In [ ]:
# ================================================================================= #
# Build a Simultaneous Translator
# ================================================================================= #

# allocate model parameters
params  = init_params(options)
params  = load_params(model, params)
tparams = init_tparams(params)

# print 'build the model for computing cost (full source sentence).'
trng, use_noise, \
_x, _x_mask, _y, _y_mask, \
opt_ret, \
cost, f_cost = build_model(tparams, options)
print 'done'

# functions for sampler
f_sim_ctx, f_sim_init, f_sim_next = build_simultaneous_sampler(tparams, options, trng)

# check the ID:
policy['base'] = _model
_policy = Policy(trng, options, policy, config,
                 n_in=options['readout_dim'] + 1 if config['coverage'] else options['readout_dim'],
                 n_out=3 if config['forget'] else 2,
                 recurrent=policy['recurrent'], id=id)

print 'build everything ok.'

In [ ]:
# make the dataset ready for training & validation
trainIter = TextIterator(options['datasets'][0], options['datasets'][1],
                         options['dictionaries'][0], options['dictionaries'][1],
                         n_words_source=options['n_words_src'], n_words_target=options['n_words'],
                         batch_size=config['batchsize'],
                         maxlen=options['maxlen'])
train_num = trainIter.num

valid_b   = 1
validIter = TextIterator(options['valid_datasets'][0], options['valid_datasets'][1],
                         options['dictionaries'][0], options['dictionaries'][1],
                         n_words_source=options['n_words_src'], n_words_target=options['n_words'],
                         batch_size=valid_b, cache=1,
                         maxlen=1000000)

valid_num = validIter.num

valid_ = options['valid_datasets'][0]
valid_num = _check_length(valid_)
print 'training set {} lines / validation set {} lines'.format(train_num, valid_num)
print 'use the reward function {}'.format(chr(config['Rtype'] + 65))


In [ ]:
# load the pretrained policy
# modelname = '160921-195056'
# iters     = '47000'
# modelname = '160914-140602'
# iters     = '52800'
# modelname = '160921-195108'
# iters     = '34000'
# modelname = '160916-232539'
# iters     = '61400'
# modelname = '160921-195220'
# iters     = 34600
# modelname = '160916-232619'
# iters     = '105400'
modelname   = '160925-000643'
iters       = '24400'

trained_policy = '{}-{}.iter={}.npz'.format(modelname, _model, iters)
if not os.path.exists('.policy/{}'.format(trained_policy)):
    print 'download the model from HPC server'                                                                                       
    os.system('scp jg5223@mercer:/scratch/jg5223/exp/.policy/{} .policy/'.format(trained_policy)) 


r = numpy.load('.policy/' + trained_policy)
for p in _policy.tparams:
    _policy.tparams[p].set_value(r[p])
for p in _policy.tparams_b:
    _policy.tparams_b[p].set_value(r[p])
   
print 'load trained policy ok'

In [ ]:
# # load the pretrained policy
# modelname = '160921-195056'
# iters     = '47000'
# modelname = '160914-140602'
# iters     = '52800'
# modelname = '160921-195108'
# iters     = '34000'
# modelname = '160916-232539'
# iters     = '61400'
# modelname = '160921-195220'
# iters     = 34600
# # modelname = '160916-232619'
# # iters     = '105400'

# trained_policy = '.policy/{}-{}.iter={}.npz'.format(modelname, _model, iters)

# r = numpy.load(trained_policy)
# for p in _policy.tparams:
#     _policy.tparams[p].set_value(r[p])
# for p in _policy.tparams_b:
#     _policy.tparams_b[p].set_value(r[p])
   
# print 'load trained policy ok'

from insepection import *
import shutil

# ================================================================================= #
# Main Loop: Run
# ================================================================================= #
def _translate(src, trg, train=False, samples=config['sample'], greedy=False):
    ret = simultaneous_decoding(
        f_sim_ctx, f_sim_init,
        f_sim_next, f_cost,
        _policy,
        src, trg, word_idict_trg,
        step=config['step'], peek=config['peek'], sidx=config['s0'],
        n_samples=samples,
        reward_config={'target': config['target'],
                       'gamma':  config['gamma'],
                       'Rtype':  config['Rtype'],
                       'maxsrc': config['maxsrc'],
                       'greedy': greedy,
                       'upper':  config['upper']},
        train=train,
        use_forget=config['forget'],
        use_newinput=config['pre'],
        use_coverage=config['coverage'],
        on_groundtruth=0 if config['finetune'] == 'nope' else 10, 
        full_attention=True)
    return ret

print 'Start Simultaneous Translator...'

tid           = '_greedy_{}'.format(modelname)
action_space  = ['W', 'C', 'F']
Log_avg       = {}
time0         = timer()
pipe          = PIPE(['x', 'x_mask', 'y', 'y_mask', 'c_mask'])

# for validation
# doing the whole validation!!
reference  = []
system     = []
refbpe     = []
srcbpe     = []
actions    = []

print 'start validation'

collections = [[], [], [], [], []]
probar_v = Progbar(valid_num / valid_b + 1)
validIter.reset()
for ij, (srcs, trgs) in enumerate(validIter):

    statistics = _translate(srcs, trgs, train=False, samples=1, greedy=True)

    src = [w.replace('%', '$%$') + ' $<eos>$' for w in _seqs2words(statistics['SWord'], word_idict)]
    trs = [w.replace('%', '$%$') for w in statistics['Words']]

    act = statistics['action']
    att = statistics['attentions']
    ott = statistics['old_attend']
    
    # print len(src[0]), len(trs[0]), len(act[0]), len(att[0]), len(ott[0])  
    quality, delay, reward = zip(*statistics['track']) 
    
    reference += statistics['Ref']
    system    += statistics['Sys']
    
    #print statistics['SWord']
    srcbpe    += _seqs2words(statistics['SWord'], word_idict)
    refbpe    += statistics['Words']
    actions   += statistics['action']
      
    # compute the average consective waiting length
    def _consective(action):
        waits = []
        temp = 0
        for a in action:
            if a == 0:
                temp += 1
            elif temp > 0:
                waits += [temp]
                temp = 0

        if temp > 0:
            waits += [temp]

        mean = numpy.mean(waits)
        gec = numpy.max(waits)  # numpy.prod(waits) ** (1./len(waits))
        return mean, gec

    def _max_length(action):
        _cur = 0
        _end = 0
        _max = 0
        for it, a in enumerate(action):
            if a == 0:
                _cur += 1
            elif a == 2:
                _end += 1

            temp = _cur - _end
            if temp > _max:
                _max = temp
        return _max

    maxlen = [_max_length(action) for action in statistics['action']]
    means, gecs = zip(*(_consective(action) for action in statistics['action']))

    collections[0] += quality
    collections[1] += delay
    collections[2] += means
    collections[3] += gecs
    collections[4] += maxlen

    #print '\n'
    #print src
    #print trs
    infoo = 'ID={}_(BLEU={}_AP={}_CW={})'.format(ij, numpy.mean(quality), numpy.mean(delay), numpy.mean(means))
    try:
        heatmap2(src, None, trs, act, 0, att, ott, name=modelname, info={'index': infoo})
        shutil.copy('.images/M_{}/Idx={}||.pdf'.format(modelname, infoo), 'heatmaps/M_{}/'.format(modelname))
    except Exception:
        print 'i am sorry, cannot plot.'
        
        
    print '\n'
    
    
    values = [('quality', numpy.mean(quality)), ('delay', numpy.mean(delay)),
              ('wait_mean', numpy.mean(means)), ('wait_max', numpy.mean(gecs)),
              ('max_len', numpy.mean(maxlen))]
    probar_v.update(ij + 1, values=values)


validIter.reset()
valid_bleu, valid_delay, valid_wait, valid_wait_gec, valid_mx = [numpy.mean(a) for a in collections]
print 'AVG BLEU = {}, DELAY = {}, WAIT(MEAN) = {}, WAIT(MAX) = {}, MaxLen={}'.format(
    valid_bleu, valid_delay, valid_wait, valid_wait_gec, valid_mx)

print len(system), len(reference)
print 'Compute the Corpus BLEU={} (greedy)'.format(corpus_bleu(reference, system))


import sys; sys.exit()

# with open(WORK + '.translate/test-{}.txt'.format(tid), 'w') as fout:
#     for sys in system:
#         fout.write('{}\n'.format(' '.join(sys)))

# with open(WORK + '.translate/ref-{}.txt'.format(tid), 'w') as fout:
#     for ref in reference:
#         fout.write('{}\n'.format(' '.join(ref[0])))

with open(WORK + 'translate/analysis-{}.txt'.format(tid), 'w') as fout:
    for iddd, (src, ref, acts) in enumerate(zip(srcbpe, refbpe, actions)):
        fout.write(src + '\n')
        fout.write(ref + '\n')
        fout.write(' '.join([str(a) for a in acts]) + '\n')
        
        q, d, m = collections[0][iddd], collections[1][iddd], collections[2][iddd] 
        fout.write('{} {} {}\n'.format(q, d, m))
    
print 'write down'
    

In [ ]:
# here we need to write a srcipt to compare translation results of different
# targets
import os
import numpy
import sys
from utils import Progbar

data  = dict()
trans = dict()
trans['source'] = []
for parent, folders, files in os.walk('translate'):
    for file in files:
        if   '140602' in file:
            name = 'ap3'
        elif '195108' in file:
            name = 'ap5'
        elif '195056' in file:
            name = 'ap7'
        elif '232539' in file:
            name = 'cw2'
        elif '195220' in file:
            name = 'cw5'
        elif '232619' in file:
            name = 'cw8'
        data[name] = dict()
        
        with open(os.path.join(parent, file)) as f:
            src = f.readline().strip()
            while src:
                if src not in trans['source']:
                    trans['source'].append(src)
                    
                trg  = f.readline().strip()
                act  = f.readline().strip()
                scc  = f.readline().strip()
                data[name][src] = [trg, act, scc]
                
                src  = f.readline().strip()
names = []
for name in data:
    trans[name] = []
    names.append(name)
    
for src in trans['source']:
    for name in data:
        trans[name].append(data[name][src])
                  
print 'done.'                

results = open('compare21.txt', 'w')
probar  = Progbar(len(trans['source']))

for itt, src in enumerate(trans['source']):
    srcs = src.split() + ['<eos>']
    decs = [{name: [] for name in names} for _ in srcs]
    
    show = ''
    bleu = []
    for name in data:
        trgs, acts, scores = [w.split() for w in trans[name][itt]]
        trgs += ['<eos>']
        sid = -1
        tid = 0
        for a in acts:
            if a == '0':
                sid += 1
            if a == '1':
                decs[sid][name].append(trgs[tid])
                tid += 1
        show += '{} (B {:.4}/A {:.4}/C {:.4}) ||'.format(name, scores[0], scores[1], scores[2])
        bleu += [float(scores[0])]
        
    if numpy.max(bleu) < 0.5:
        continue
    if numpy.min(bleu) < 0.3:
        continue
    if not((bleu[1] < bleu[5]) and (bleu[5] < bleu[4])):
        continue
        
    show += '\n'
    show += 'source ||\t'
    for name in names:
        show += (name + '|\t')
    show += '\n'
    
    for iss, s in enumerate(srcs):
        show += (s + '||\t')
        for name in names:
            ts = ' '.join(decs[iss][name])
            show += (ts + '|\t')
        show += '\n'
    show += '**********' * 10 + '\n'
    results.write(show + '\n')
    probar.update(itt)


results.close()
print '\ndone'   
    
    
            